# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging
from fastoad.utils.postprocessing import VariableViewer
from models.register import register_openmdao_systems
import shutil
register_openmdao_systems()

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, 'oad_process_drone.toml')
SOURCE_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In the configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the reference parameters from my_ref_values.xml as default values:

In [2]:
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)
#api.generate_inputs(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )
INFO    : Problem inputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_inputs.xml


You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [3]:
api.list_variables(CONFIGURATION_FILE)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )


,I/O,Name,Description
0,IN,data:ESC:efficiency,
1,IN,data:ESC:reference:mass,
2,IN,data:ESC:reference:power,
3,IN,data:ESC:reference:voltage,
4,IN,data:ESC:settings:k_ESC,
5,IN,data:battery:cell:voltage:estimated,
6,IN,data:battery:discharge_limit,
7,IN,data:battery:reference:capacity,
8,IN,data:battery:reference:current:max,
9,IN,data:battery:reference:mass,


Another useful feature is to list the modules of the model defined in the configuration file:

In [4]:
api.list_systems(CONFIGURATION_FILE)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )
INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )


== AVAILABLE SYSTEM IDENTIFIERS ====================================================================
----------------------------------------------------------------------------------------------------
  IDENTIFIER:   energy.battery
  PATH:         D:\THESE\Tools\FAST_OAD_Drone\models\Energy_source\energy_source.py
  DOMAIN:       Unspecified
  DESCRIPTION:  
    Group containing the Battery MDA.

----------------------------------------------------------------------------------------------------
  IDENTIFIER:   fastoad.aerodynamics.highspeed.legacy
  PATH:         D:\Softwares\Anaconda3\envs\Drone_OAD\lib\site-packages\fastoad\models\aerodynamics\aerodynamics_high_speed.py
  DOMAIN:       Aerodynamics
  DESCRIPTION:  
    Computes aerodynamic polar of the aircraft in cruise conditions.

    Drag contributions of each part of the aircraft are computed though analytical
    models.

----------------------------------------------------------------------------------------------------
  ID

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [5]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )
INFO    : N2 diagram written in D:\THESE\Tools\FAST_OAD_Drone\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [6]:
# XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
# api.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
# from IPython.display import IFrame
# IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [7]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api.variable_viewer(INPUT_FILE)

In [8]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )
INFO    : Computation finished after 0.09 seconds
INFO    : Problem outputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_outputs.xml


NL: NLBGS Converged in 2 iterations


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [9]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mda.xml')
shutil.copy(OUTPUT_FILE, MDA_OUTPUT_FILE)

'workdir\\problem_outputs_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [10]:
api.variable_viewer(OUTPUT_FILE)

## 3. Running your first MDO


In [11]:
# api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file.

In [12]:
api.optimization_viewer(CONFIGURATION_FILE)

In [20]:
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:\THESE\Tools\FAST_OAD_Drone\data\../models
INFO    : Installed bundle models.register (ID 32 )
INFO    : Installed bundle models (ID 31 )


NL: NLBGS Converged in 2 iterations
NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 2 iterations
NL: NLBGS Converged in 2 iterations
NL: NLBGS Converged in 2 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [1.06827496]
            Iterations: 4
            Function evaluations: 4
            Gradient evaluations: 4
Optimization Complete
-----------------------------------


INFO    : Computation finished after 0.27 seconds
INFO    : Problem outputs written in D:\THESE\Tools\FAST_OAD_Drone\workdir\problem_outputs.xml


Let's save these results:

In [14]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')
shutil.copy(OUTPUT_FILE, OPT_OUTPUT_FILE)

'workdir\\problem_outputs_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [15]:
api.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [16]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')

In [17]:
api.variable_viewer(RESULT_FILE)